first of we start with preprocessing the raw data

In [4]:
import pandas as pd
import numpy as np
import scipy

In [5]:
df=pd.read_csv('../data/RAW_interactions.csv')
df.drop('review', axis=1, inplace=True)
df.drop('date', axis=1, inplace=True)
df.reset_index()
df.drop_duplicates(subset=['user_id', 'recipe_id'])

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5
...,...,...,...
1132362,116593,72730,0
1132363,583662,386618,5
1132364,157126,78003,5
1132365,53932,78003,4


The recipes and users don't go from 0 to amount so if we were to put this in a matrix we would get empty columns and rows. This is not that handy so we reindex both the user_id and recipe_ids

This is a step we must not forget when entering the data in the model, as we also need to remap our input data using the same remapping that was used here

In [6]:
userSet=set(df['user_id'].to_list())
user_transform_dict=dict(map(reversed, enumerate(userSet)))
recipeSet=set(df['recipe_id'].to_list())
recipe_transform_dict=dict(map(reversed, enumerate(recipeSet)))

In [7]:
keep_nan = [k for k,v in user_transform_dict.items() if pd.isnull(v)]
tochange=df['user_id']
df['user_id'] = tochange.map(user_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan)))

keep_nan = [k for k,v in recipe_transform_dict.items() if pd.isnull(v)]
tochange=df['recipe_id']
df['recipe_id'] = tochange.map(recipe_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan)))

Creation of the folds for k-fold validation

In [8]:
unique_users=df.user_id.unique()

unique_users.sort(axis=0)

In [9]:
from sklearn.model_selection import train_test_split,KFold
k=10
kf = KFold(n_splits=k)
kf.get_n_splits(df)
folds=list()
for train_index, test_index in kf.split(unique_users):
    X_train=df.query('user_id in @train_index')
    X_test=df.query('user_id in @test_index')
    folds.append((X_train,X_test))

Here starts the actual programming of the model

In [10]:
class Easer:
    def __init__(self):
        pass

    def train(self, X_train, lambda_= 0.5):
        self.X = X_train

        #X_train_t=X_train.copy().transpose()
        G= X_train.T.dot(X_train) # sparse
        #G= X_train.T.dot(X_train).toarray() # dense
        diagIndices = np.diag_indices(G.shape[0])
        G[diagIndices] += lambda_
        P = scipy.sparse.linalg.inv(G) # sparse
        #P = np.linalg.inv(G) # dense
        self.B = P/(-np.diag(P))
        self.B[diagIndices] = 0

        self.pred = self.X*self.B

    def predicts(self,xu):
        #TODO give back the predicted vector given a user interaction vector
        return xu*self.B


In [ ]:
from scipy import sparse
#for fold in folds: # removed for loop for testing
train_data=folds[0][0]
ratings=train_data.rating
idx=(train_data.user_id,train_data.recipe_id)
#Here we have the user item matrix
X_train=sparse.csc_matrix((ratings,idx),shape=(len(df.user_id.unique()),len(df.recipe_id.unique())), dtype=float)
#train model
model = Easer()
model.train(X_train)
print("done")
#break

